<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/deep_learning_for_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning for text
In this notebook we will use deep learning algorithms to perform some tasks on text. Common tasks are: text classification, content filtering, sentiment analysis, translation, text summarization, language modeling. Since deep learning models are differentiable functions that can only process tensors of numbers we have to transform text into numerical tensors. The steps to transform text into numbers are

1. Text standardization
2. Tokenization
3. Convert the tokens into numerical arrays

In the first step we perform the same kind of transformations used to build a search engine: lower case, remove punctuation, word stemming. The words left represent the tokens, elements of a "clean" vocabulary. We can also build token made of two words, called bigram, or more words called N-gram. The number of tokens N defines the dimensionality of a space where each token represents a dimension and a text can be represented as vector in such space. We can define a metric is such a space so that we can measure the distance between two sequences of tokens. Depending on the task at hand, we might need to process the tokens in the order in which they appear in the text. In this case we will build a sequence model. If our task doesn't need the order of the tokens we will build a bag-of-words model.

## Bag-of-words models
We will use the [IMDB dataset]('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) to train our bag-of-words model for sentiment analysis, that is a kind of text classification. The dataset contains 50k movie reviews. We download the dataset and extract the files into a folder. The dataset contains two subfolders train/ and test/ each containing 25k reviews split into two subfolders pos/ and neg/ with 12500 txt files. Each file contains a short text, the content of the review. The name of the file is created from the review's unique identifier and the score given to the movie. A score equal or higher than 7 is positive, a score equal or lower than 4 is negative. This same example is available for PyTorch in [another repository](https://github.com/luigiselmi/machine_learning_notes/blob/main/pml3/sentiment_analysis.ipynb).

In [24]:
!wget -nv 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' -P './data'

2024-10-28 12:42:11 URL:https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz [84125825/84125825] -> "./data/aclImdb_v1.tar.gz.1" [1]


In [ ]:
!tar -xf data/aclImdb_v1.tar.gz -C data/

In [ ]:
!rm -r data/aclImdb/train/unsup

We print the content of one review

In [ ]:
!cat data/aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

We use 20% of the training data for validation

In [ ]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("data/aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

We create three datasets for train, validation and test using the utility function text_dataset_from_directory() that returns a [Dataset](https://keras.io/api/data_loading/text/) object that asynchronouly fetches the data from a source data folder that has a structure like

main_directory/  
...class_a/  
......a_text_1.txt  
......a_text_2.txt  
...class_b/  
......b_text_1.txt  
......b_text_2.txt  

Each dataset contains a number of batches with 32 reviews with the respective numerical labels starting from 0 for the first class, in our case negative review, and 1 for the positive reviews.

In [ ]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory('data/aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory('data/aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('data/aclImdb/test', batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
print('Number of batches for training: {:d}'.format(len(train_ds)))
print('Number of batches for validation: {:d}'.format(len(val_ds)))
print('Number of batches for test: {:d}'.format(len(test_ds)))

Number of batches for training: 625
Number of batches for validation: 157
Number of batches for test: 782


In [ ]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"The tourist season has just ended on a remote island off the coast of Scotland, winter is beginning to set in and the inhabitants, both humans and sheep alike are settling down to much quieter times ahead. Michael Gaffikin (James Warwick) a former paratrooper in the British Army, is the local dentist, he's not an islander by birth and as such his relationship with local artist and cartographer Fiona Patterson (Celia Imrie) is always being viewed with a little suspicion, not maliciously, but just out of the protective instincts the tight knit community have for their kin. The islands serenity is broken when Gaffikin out for a solitary round of golf finds the headless remains of a brutally slain woman. He immediately reports his gruesome find to Insp Inskip(Maurice Ro\xc3\xabves) at the islands police station, Inskip arranges for delivery of the remains to local

We use the set of the most common 20000 tokens with only one word, aka unigrams, to define a space of the same dimensionality to represent each review as a vector in this space. We use the TextVectoization module to return a vector from a review.

In [ ]:
from keras.layers import TextVectorization
text_vectorization = TextVectorization(max_tokens=20000, output_mode="multi_hot",)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

We define a simple neural network model with 4 layers: an input layer of size 20000, a fully connected layer, a dropout layer, and finally an output layer with only one unit. The input data are text reviews that have been preprocessed so that each review is represented as a vector in the input space and an integer, 0 or 1, to represent its class.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation="relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation="sigmoid")(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop",
  loss="binary_crossentropy",
  metrics=["accuracy"])
  return model

In [ ]:
model = get_model()
model.summary()
callbacks = [
  keras.callbacks.ModelCheckpoint("binary_1gram.keras",
  save_best_only=True)
]

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 515s 815ms/step - accuracy: 0.7742 - loss: 0.4874 - val_accuracy: 0.8862 - val_loss: 0.2930
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 50s 3ms/step - accuracy: 0.8935 - loss: 0.2761 - val_accuracy: 0.8954 - val_loss: 0.2765
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9129 - loss: 0.2357 - val_accuracy: 0.8938 - val_loss: 0.2903
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9241 - loss: 0.2192 - val_accuracy: 0.8946 - val_loss: 0.3075
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9320 - loss: 0.2081 - val_accuracy: 0.8916 - val_loss: 0.3247
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9339 - loss: 0.2036 - val_accuracy: 0.8928 - val_loss: 0.3369
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9378 - loss: 0.2008 - val_accuracy: 0.8896 - val_loss: 0.3442
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9379 - loss: 0.2009 - val_accurac

We can see that our simple model with unigrams as tokens is able to achieve 88% accuracy in the text classification task, not bad for such a simple model and data representation.

In [ ]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 504s 643ms/step - accuracy: 0.8899 - loss: 0.2806
Test acc: 0.887


Now we use bigrams instead of unigrams to build our data space. Again we use only 20000 bigrams in order to keep the size of the space like before.

In [ ]:
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode="multi_hot",)
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
model = get_model()
model.summary()
callbacks = [
  keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)
]

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(binary_2gram_train_ds.cache(),
  validation_data=binary_2gram_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1072s 2s/step - accuracy: 0.7878 - loss: 0.4576 - val_accuracy: 0.8986 - val_loss: 0.2675
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - accuracy: 0.9103 - loss: 0.2493 - val_accuracy: 0.9062 - val_loss: 0.2656
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9311 - loss: 0.1922 - val_accuracy: 0.9068 - val_loss: 0.2828
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9430 - loss: 0.1806 - val_accuracy: 0.9056 - val_loss: 0.3013
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9505 - loss: 0.1637 - val_accuracy: 0.9028 - val_loss: 0.3205
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9532 - loss: 0.1642 - val_accuracy: 0.9030 - val_loss: 0.3328
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9560 - loss: 0.1562 - val_accuracy: 0.9002 - val_loss: 0.3545
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9581 - loss: 0.1489 - val_accuracy:

We can see that with bigrams out model improves its performance on the text classification task achieving 90% accuracy.

In [ ]:
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 1045s 1s/step - accuracy: 0.9014 - loss: 0.2635
Test acc: 0.900


We now use bigrams with the number of times each of them appears in the review normalized to the number of documents in which they appear, the so called TF-IDF normalization.

In [ ]:
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode="tf_idf",)

In [ ]:
text_vectorization.adapt(text_only_train_ds)
text_vectorization.adapt(text_only_train_ds)
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
model = get_model()
model.summary()
callbacks = [
  keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
  save_best_only=True)
]

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(tfidf_2gram_train_ds.cache(),
  validation_data=tfidf_2gram_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1057s 2s/step - accuracy: 0.7146 - loss: 0.8313 - val_accuracy: 0.8708 - val_loss: 0.3246
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8686 - loss: 0.3313 - val_accuracy: 0.8918 - val_loss: 0.3158
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8851 - loss: 0.2876 - val_accuracy: 0.8834 - val_loss: 0.3081
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8953 - loss: 0.2637 - val_accuracy: 0.8872 - val_loss: 0.3221
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9011 - loss: 0.2478 - val_accuracy: 0.8816 - val_loss: 0.3252
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9049 - loss: 0.2344 - val_accuracy: 0.8860 - val_loss: 0.3547
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9086 - loss: 0.2242 - val_accuracy: 0.8840 - val_loss: 0.3214
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9111 - loss: 0.2167 - val_accuracy: 

In [ ]:
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 1027s 1s/step - accuracy: 0.8856 - loss: 0.3131
Test acc: 0.880
